## Train a Multi agent system using RLLIB

This notebook will demonstrate how to use the `PrimaiteRayMARLEnv` to train a very basic system with two PPO agents.

#### First, Import packages and read our config file.

In [ ]:
from primaite.game.game import PrimaiteGame
import yaml

from primaite.session.environment import PrimaiteRayEnv
from primaite import PRIMAITE_PATHS

import ray
from ray import air, tune
from ray.rllib.algorithms.ppo import PPOConfig
from primaite.session.environment import PrimaiteRayMARLEnv

# If you get an error saying this config file doesn't exist, you may need to run `primaite setup` in your command line
# to copy the files to your user data path.
with open(PRIMAITE_PATHS.user_config_path / 'example_config/example_config_2_rl_agents.yaml', 'r') as f:
    cfg = yaml.safe_load(f)

ray.init(local_mode=True)

#### Create a Ray algorithm config which accepts our two agents

In [ ]:
config = (
    PPOConfig()
    .multi_agent(
        policies={'defender_1','defender_2'}, # These names are the same as the agents defined in the example config.
        policy_mapping_fn=lambda agent_id, episode, worker, **kw: agent_id,
        )
    .environment(env=PrimaiteRayMARLEnv, env_config={"cfg":cfg})#, disable_env_checking=True)
    .rollouts(num_rollout_workers=0)
    .training(train_batch_size=128)
    )


#### Set training parameters and start the training
This example will save outputs to a default Ray directory and use mostly default settings.

In [ ]:
tune.Tuner(
    "PPO",
    run_config=air.RunConfig(
        stop={"timesteps_total": 511},
    ),
    param_space=config
).fit()